In [34]:
from mlflow import MlflowClient
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

TRACKING_URI="http://mlflow.mlnow.frenoid.com:30080/"

In [36]:
# Credentials are stored in ~/.mlflow/credentials
client = MlflowClient(tracking_uri=TRACKING_URI)

In [17]:
all_experiments = client.search_experiments()

print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1708652897569, experiment_id='0', last_update_time=1708652897569, lifecycle_stage='active', name='Default', tags={}>]


In [19]:
default_experiment = [
     {"name": experiment.name, "lifecycle_stage": experiment.lifecycle_stage} for experiment in all_experiments if experiment.name == "Default"   
][0]

pprint(default_experiment)

{'lifecycle_stage': 'active', 'name': 'Default'}


In [24]:
experiment_description = (
     "This is the grocery forecasting project. "
     "This experiment contains the produce models for apples. "
)

experiment_tags = {
    "project_name": "grocery-forecasting",
    "store_dept": "produce",
    "team": "stores-ml",
    "project_quarter": "Q3-2023",
    "mlflow.note.content": experiment_description,
}

produce_apples_experiment = client.create_experiment(
    name="Apple_Models", tags=experiment_tags   
)

In [29]:
# Use search_experiments() to search on project_name tag key
apples_experiment = client.search_experiments(
    filter_string="tags.`project_name` = 'grocery-forecasting'"
)

print(vars(apples_experiment[0]))
print()
print(apples_experiment[0])

{'_experiment_id': '1', '_name': 'Apple_Models', '_artifact_location': 'mlflow-artifacts:/1', '_lifecycle_stage': 'active', '_tags': {'project_name': 'grocery-forecasting', 'store_dept': 'produce', 'team': 'stores-ml', 'project_quarter': 'Q3-2023', 'mlflow.note.content': 'This is the grocery forecasting project. This experiment contains the produce models for apples. '}, '_creation_time': 1708783225351, '_last_update_time': 1708783225351}

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1708783225351, experiment_id='1', last_update_time=1708783225351, lifecycle_stage='active', name='Apple_Models', tags={'mlflow.note.content': 'This is the grocery forecasting project. This '
                        'experiment contains the produce models for apples. ',
 'project_name': 'grocery-forecasting',
 'project_quarter': 'Q3-2023',
 'store_dept': 'produce',
 'team': 'stores-ml'}>


In [31]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


def generate_apple_sales_data_with_promo_adjustment(
    base_demand: int = 1000, n_rows: int = 5000
):
    """
    Generates a synthetic dataset for predicting apple sales demand with seasonality
    and inflation.

    This function creates a pandas DataFrame with features relevant to apple sales.
    The features include date, average_temperature, rainfall, weekend flag, holiday flag,
    promotional flag, price_per_kg, and the previous day's demand. The target variable,
    'demand', is generated based on a combination of these features with some added noise.

    Args:
        base_demand (int, optional): Base demand for apples. Defaults to 1000.
        n_rows (int, optional): Number of rows (days) of data to generate. Defaults to 5000.

    Returns:
        pd.DataFrame: DataFrame with features and target variable for apple sales prediction.

    Example:
        >>> df = generate_apple_sales_data_with_seasonality(base_demand=1200, n_rows=6000)
        >>> df.head()
    """

    # Set seed for reproducibility
    np.random.seed(9999)

    # Create date range
    dates = [datetime.now() - timedelta(days=i) for i in range(n_rows)]
    dates.reverse()

    # Generate features
    df = pd.DataFrame(
        {
            "date": dates,
            "average_temperature": np.random.uniform(10, 35, n_rows),
            "rainfall": np.random.exponential(5, n_rows),
            "weekend": [(date.weekday() >= 5) * 1 for date in dates],
            "holiday": np.random.choice([0, 1], n_rows, p=[0.97, 0.03]),
            "price_per_kg": np.random.uniform(0.5, 3, n_rows),
            "month": [date.month for date in dates],
        }
    )

    # Introduce inflation over time (years)
    df["inflation_multiplier"] = (
        1 + (df["date"].dt.year - df["date"].dt.year.min()) * 0.03
    )

    # Incorporate seasonality due to apple harvests
    df["harvest_effect"] = np.sin(2 * np.pi * (df["month"] - 3) / 12) + np.sin(
        2 * np.pi * (df["month"] - 9) / 12
    )

    # Modify the price_per_kg based on harvest effect
    df["price_per_kg"] = df["price_per_kg"] - df["harvest_effect"] * 0.5

    # Adjust promo periods to coincide with periods lagging peak harvest by 1 month
    peak_months = [4, 10]  # months following the peak availability
    df["promo"] = np.where(
        df["month"].isin(peak_months),
        1,
        np.random.choice([0, 1], n_rows, p=[0.85, 0.15]),
    )

    # Generate target variable based on features
    base_price_effect = -df["price_per_kg"] * 50
    seasonality_effect = df["harvest_effect"] * 50
    promo_effect = df["promo"] * 200

    df["demand"] = (
        base_demand
        + base_price_effect
        + seasonality_effect
        + promo_effect
        + df["weekend"] * 300
        + np.random.normal(0, 50, n_rows)
    ) * df[
        "inflation_multiplier"
    ]  # adding random noise

    # Add previous day's demand
    df["previous_days_demand"] = df["demand"].shift(1)
    df["previous_days_demand"].fillna(
        method="bfill", inplace=True
    )  # fill the first row

    # Drop temporary columns
    df.drop(columns=["inflation_multiplier", "harvest_effect", "month"], inplace=True)

    return df

In [43]:
# Generate the dataset!

data = generate_apple_sales_data_with_promo_adjustment(base_demand=1_000, n_rows=1_000)

data[-20:]

/tmp/ipykernel_20393/3859398416.py:89: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["previous_days_demand"].fillna(


,date,average_temperature,rainfall,weekend,holiday,price_per_kg,promo,demand,previous_days_demand
980,2024-02-05 15:30:49.992394,34.130183,1.454065,0,0,1.449177,0,999.306290,1356.418398
981,2024-02-06 15:30:49.992393,32.353643,9.462859,0,0,2.856503,0,842.129427,999.306290
982,2024-02-07 15:30:49.992393,18.816833,0.391470,0,0,1.326429,0,990.616709,842.129427
983,2024-02-08 15:30:49.992392,34.533012,2.120477,0,0,0.970131,0,1068.802075,990.616709
984,2024-02-09 15:30:49.992392,23.057202,2.365705,0,0,1.049931,0,1019.486305,1068.802075
985,2024-02-10 15:30:49.992391,34.810165,3.089005,1,0,2.035149,0,1329.564672,1019.486305
986,2024-02-11 15:30:49.992391,29.208905,3.673292,1,0,2.518098,0,1413.143402,1329.564672
987,2024-02-12 15:30:49.992390,16.428676,4.077782,0,0,1.268979,0,1093.207186,1413.143402
988,2024-02-13 15:30:49.992389,32.067512,2.734454,0,0,0.762317,0,1069.939894,1093.207186
989,2024-02-14 15:30:49.992389,31.938203,13.883486,0,0,1.153301,0,994.409540,1069.939894


In [37]:
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [39]:
mlflow.set_tracking_uri(TRACKING_URI)

In [41]:
# Set the current active experiment to "Apple_Models" experiment and returns the experiment metadata
apple_experiment = mlflow.set_experiment("Apple_Models")

# Define a run name for this iteration of training, otherwise a unique name will be auto-generated for your run
run_name = "apples_rf_test"

# Define an artifact path that the model will be saved to
artifact_path = "rf_apples"

In [46]:
# Split the data into features and target and drop irrelevant date field and target field
X = data.drop(columns=["date", "demand"])
y = data["demand"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    "n_estimators": 100,
    "max_depth": 6,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "bootstrap": True,
    "oob_score": False,
    "random_state": 888,
}

# Train the RandomForestRegressor
rf = RandomForestRegressor(**params)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf.predict(X_val)

# Calculate error metrics
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

# Assemble the metrics we're going to write into a collection
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

# Initiate the MLFlow run context
with mlflow.start_run(run_name=run_name) as run:
    # Log the parameters used for the model fit
    mlflow.log_params(params)
    
    # Log the error metrics that were calculated during validation
    mlflow.log_metrics(metrics)
    
    # Log an instance of the trained model for later use
    mlflow.sklearn.log_model(
        sk_model=rf, input_example=X_val, artifact_path=artifact_path
    )

/opt/conda/envs/python3-11/lib/python3.11/site-packages/mlflow/models/signature.py:358: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  input_schema = _infer_schema(input_example)
/opt/conda/envs/python3-11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
